In [2]:
import pandas as pd
from sklearn.neighbors import NearestNeighbors

In [3]:
# Function to reduce the memory usage of a DataFrame.
def reduce_memory(df):
    for col in df.columns:
        if df[col].dtype == 'float64':
            df[col] = df[col].astype('float32')
        if df[col].dtype == 'int64':
            df[col] = df[col].astype('int32')
    return df

# Generator function to load data in chunks.
def data_generator(df, chunksize=10000):
    for i in range(0, df.shape[0], chunksize):
        yield df.iloc[i:i+chunksize]

In [4]:
df = reduce_memory(pd.read_csv("data/Dataset.csv"))

In [5]:
df.head()

,UserID,Game,purchase/play,Heure_jouee,AppID,Release date,Estimated owners,Peak CCU,Required age,Price,...,Median playtime forever,Median playtime two weeks,Developers,Publishers,Categories,Genres,Tags,Score,Recommandable,GameID
0,1,The Elder Scrolls V Skyrim,play,273.0,72850,"Nov 10, 2011",5000000 - 10000000,4383,17,19.99,...,50.533333,14.633333,Bethesda Game Studios,Bethesda Softworks,"Single-player,Steam Achievements,Steam Trading...",RPG,"Open World,RPG,Fantasy,Adventure,Dragons,Modda...",5.0,True,1
1,2,The Elder Scrolls V Skyrim,play,58.0,72850,"Nov 10, 2011",5000000 - 10000000,4383,17,19.99,...,50.533333,14.633333,Bethesda Game Studios,Bethesda Softworks,"Single-player,Steam Achievements,Steam Trading...",RPG,"Open World,RPG,Fantasy,Adventure,Dragons,Modda...",5.0,True,1
2,3,The Elder Scrolls V Skyrim,play,110.0,72850,"Nov 10, 2011",5000000 - 10000000,4383,17,19.99,...,50.533333,14.633333,Bethesda Game Studios,Bethesda Softworks,"Single-player,Steam Achievements,Steam Trading...",RPG,"Open World,RPG,Fantasy,Adventure,Dragons,Modda...",5.0,True,1
3,4,The Elder Scrolls V Skyrim,play,465.0,72850,"Nov 10, 2011",5000000 - 10000000,4383,17,19.99,...,50.533333,14.633333,Bethesda Game Studios,Bethesda Softworks,"Single-player,Steam Achievements,Steam Trading...",RPG,"Open World,RPG,Fantasy,Adventure,Dragons,Modda...",5.0,True,1
4,5,The Elder Scrolls V Skyrim,play,220.0,72850,"Nov 10, 2011",5000000 - 10000000,4383,17,19.99,...,50.533333,14.633333,Bethesda Game Studios,Bethesda Softworks,"Single-player,Steam Achievements,Steam Trading...",RPG,"Open World,RPG,Fantasy,Adventure,Dragons,Modda...",5.0,True,1


In [ ]:
from scipy.sparse import coo_matrix

# map each user and item to a unique numeric value
user_ids = df['UserID'].astype('category').cat.codes
item_ids = df['GameID'].astype('category').cat.codes

# Get the unique user and game ids
unique_user_ids = df['UserID'].astype('category').cat.categories
unique_item_ids = df['GameID'].astype('category').cat.categories

# create a sparse matrix
user_game_matrix = coo_matrix((df['Heure_jouee'], (user_ids, item_ids)))

# Fit the model
model_knn = NearestNeighbors(metric='cosine', algorithm='brute')
model_knn.fit(user_game_matrix)

# Get top 5 recommendations for first user
distances, indices = model_knn.kneighbors(user_game_matrix.getrow(0), n_neighbors=6)
recommended_users = [unique_user_ids[i] for i in indices.flatten()[1:]]
print(f'Recommended users for the first user are: {recommended_users}')

In [ ]:
## On analyse le UserID n°1

# df[df["UserID"] == 1]

## L'algo nous indique que les UserID les plus proches sont les 5 suivants

# df[df["UserID"] == 332]
# df[df["UserID"] == 338]
# df[df["UserID"] == 598]
# df[df["UserID"] == 370]
# df[df["UserID"] == 140]

In [ ]:
# # Use a pipeline as a high-level helper
# from transformers import pipeline

# pipe = pipeline("fill-mask", model="vocab-transformers/distilbert-word2vec_256k-MLM_1M")

In [9]:
df['About the game'].head()

0    EPIC FANTASY REBORN The next chapter in the hi...
1    EPIC FANTASY REBORN The next chapter in the hi...
2    EPIC FANTASY REBORN The next chapter in the hi...
3    EPIC FANTASY REBORN The next chapter in the hi...
4    EPIC FANTASY REBORN The next chapter in the hi...
Name: About the game, dtype: object